In [ ]:
import pandas as pd
import pypsa
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import cartopy.crs as ccrs
import ipynb
import os
from os import walk
import plotly.express as px
import json
import plotly.graph_objs as gobj
import plotly.graph_objects as go
import pyproj

In [ ]:
def load_data(mypath1,filenames):
    networks = {}
    planning_horizon = []
    for file in filenames:
        year = file[-7:-3]
        print("Loading year {} ".format(year))
        networks[year] = pypsa.Network(os.path.join(mypath1,file))
        planning_horizon.append(year)
    print("I have done my deed now set me Free, mortal!")
    return(networks, planning_horizon)

In [ ]:
def get_grid_df(n,carrier):
    """Gets all capacity dataframes for the grid
    Parameters
    ----------
    n : Networks
    carrier : string (== "gas pipeline", "AC", "H2 pipeline")
        carrier (e.g. "gas pipeline", "DC") used in lines and links for transporting a specific energy
    
    Returns
    -------
    two dataframes:
    normal = classic grid, including newly build H2 grid
    retro only used for H2 retrofitted piplines.
    """
    normal = n.copy()
    retro = n.copy()
    dc = n.copy()
    if carrier == 'AC':
        normal.lines = n.lines[n.lines.type =="Al/St 240/40 4-bundle 380.0"]
        dc.links = n.links[n.links.carrier == "DC"]
        for bus in normal.buses[n.buses.carrier=="AC"].index:
            dc.buses.loc[f"{bus}", ["x","y"]] = normal.buses.loc[bus, ["x","y"]]
    else: 
        normal.links = n.links[(n.links.carrier == carrier)]
        carrier_retro = carrier + " retrofit"
        retro.links = n.links[(n.links.carrier == carrier_retro)]
        for bus in normal.buses[n.buses.carrier=="AC"].index:
            normal.buses.loc[f"{bus}"+ " "  + carrier[:3].strip(), ["x","y"]] = normal.buses.loc[bus, ["x","y"]]
            retro.buses.loc[f"{bus}"+ " " + carrier[:3].strip(), ["x","y"]] = retro.buses.loc[bus, ["x","y"]]
    return(normal,retro,dc)

In [ ]:


def get_map_object(fp):
    """loads a shapefile, simplifies it and transforms it into a, for plotly, usable format
    Parameters
    ----------
    fp : string (== ".\shape_nuts2.shx")
        fp defines a shapefile location in unicode
    
    Returns
    -------
    geopandas.geodataframe.GeoDataFrame
    """
    #read in file
    map_df = gpd.read_file(fp)
    #Returns a GeoSeries containing a simplified representation of each geometry
    map_df["geometry"] = (
        map_df.to_crs(map_df.estimate_utm_crs()).simplify(1000).to_crs(map_df.crs)
    )
    #transform to readable geo-df
    map_df.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)
    # Change CountryID for AT (and DE if need) 
    for country in range(len(map_df.COUNTRY_ID)):
        if map_df.COUNTRY_ID[country] in ['AT', 'DE']:
            map_df.COUNTRY_ID[country]= map_df.COUNTRY_ID[country] + '_' + str(map_df.id[country])
    return(map_df)

In [ ]:
fp = '.\shapefile\shapes_AT10.shx'
map_df = get_map_object(fp)

In [ ]:
mypath = r"C:\Users\johannes.misensky\OneDrive - AGGM\Dokumente\ESM_Aus\postnetworks\test_3"
mypath1 = f"{mypath}"
filenames = next(walk(mypath1), (None, None, []))[2]

networks = {}
planning_horizon = []
filenames

networks, planning_horizon = load_data(mypath1,filenames)

In [ ]:
def plot_grid(normal, retro, dc, carrier, year, button):
    """Plots a Geo. Figure in Plotly containing all clusters and a given grid
    Parameters
    ----------
    normal : Dataframe containing Links and Buses for AC and H2/gas grid, should be provided by get_grid_df funktion
    retro : Dataframe containing Links and Buses for retrofitted H2 grid, should be provided by get_grid_df funktion
    carrier : string (== "gas pipeline", "AC", "H2 pipeline")
        carrier (e.g. "gas pipeline", "AC") used to differenciate what grid to plot
    
    Returns
    -------
    Grid plot of a given carrier
    """
    # Variable to divde Capacity and therefore line thickness. 
    divisor = 1e3
    
    #plot a choropleth map
    fig = px.choropleth(map_df, geojson=map_df.geometry, 
                    locations=map_df.index, color =map_df.index ,  
                    height=500, 
                    # Colorscale atm set to one color
                    color_continuous_scale=[[0, 'rgb(229,236,246)'],
                      [0.05, 'rgb(229,236,246)'],
                      [0.1, 'rgb(229,236,246)'],
                      [0.20, 'rgb(229,236,246)'],
                      [1, 'rgb(229,236,246)']],
                    hover_name =  list(map_df.COUNTRY_ID) )

    # Formate hover_name to only show the Country ID = Cluster name
    fig.update_traces(customdata= np.stack(map_df.COUNTRY_ID, axis=0),
        hovertemplate = '<b>%{customdata}</b>' )

    #Plot grid lines and set width, AC has to be treated specially 
    if carrier == 'AC':

        for i in range(len(normal.lines.s_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    lon = [normal.buses.loc[normal.lines.bus0[i]]['x'].round(3), normal.buses.loc[normal.lines.bus1[i]]['x'].round(3)],
                    lat = [normal.buses.loc[normal.lines.bus0[i]]['y'].round(3), normal.buses.loc[normal.lines.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (normal.lines.s_nom_opt[i] * normal.lines.s_max_pu[i])/divisor,color = 'green'), #s_nom_opt nur Termische Kap = s_nom_opt * s_max_pu für net
                    hoverinfo = 'none',
                    name="{} GW AC-Grid".format(int(round(normal.lines.s_nom_opt[i]/divisor)))
                )
            )
        for i in range(len(dc.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    lon = [normal.buses.loc[normal.links.bus0[i]]['x'].round(3), normal.buses.loc[normal.links.bus1[i]]['x'].round(3)],
                    lat = [normal.buses.loc[normal.links.bus0[i]]['y'].round(3), normal.buses.loc[normal.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (dc.links.p_nom_opt[i])/divisor,color = 'red'),
                    hoverinfo = 'none',
                    opacity = 0.5,
                    name="{} GW DC-Grid".format(int(round(dc.links.p_nom_opt[i]/divisor)))
                )
            )

    else:

        for i in range(len(normal.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    lon = [normal.buses.loc[normal.links.bus0[i]]['x'].round(3), normal.buses.loc[normal.links.bus1[i]]['x'].round(3)],
                    lat = [normal.buses.loc[normal.links.bus0[i]]['y'].round(3), normal.buses.loc[normal.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (normal.links.p_nom_opt[i])/divisor,color = 'red'),
                    hoverinfo = 'none',
                    name="{} GW Pipeline".format(int(round(normal.links.p_nom_opt[i]/divisor)))

                )
            )
        for i in range(len(retro.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    lon = [normal.buses.loc[retro.links.bus0[i]]['x'].round(3), normal.buses.loc[retro.links.bus1[i]]['x'].round(3)],
                    lat = [normal.buses.loc[retro.links.bus0[i]]['y'].round(3), normal.buses.loc[retro.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (retro.links.p_nom_opt[i])/divisor,color = 'blue'),
                    hoverinfo = 'none',
                    name="{} GW retrofitted Pipeline".format(int(round(retro.links.p_nom_opt[i]/divisor)))
                )
            )

    #centralize on europe

    if button == 'europe':
        fig.update_geos(
            visible=False, resolution=50, 
            scope="europe",
            showcountries=True, countrycolor="Black",
        )
    else:
        #zoom to AT
        fig.update_geos(
            visible=False, resolution=50, 
            scope="europe",
            #center around AT
            center=dict(lon=11, lat=40), 
            projection_rotation=dict(lon=31, lat=31, roll=31),
            lataxis_range=[10,11], lonaxis_range=[33,41],
            showcountries=True, countrycolor="Black"
        )

    #Scale Figure 
    fig.update_layout(
        title=f" Grid Capacity in {year}",
        autosize=False,
        width=1000,
        height=1000,
        coloraxis_showscale=False,

    )

    #the following part is only for legend customizing

    #create empty list to store legend labels
    labels_to_show_in_legend = []

    #as far as i understand plotly needs a geoobject to exist in the plot to be filtered in legend 
    #therefor find the closest existing Capacity to 3 and 10 GW and store them in List
    for s in (10,3):
        if carrier == 'AC':
            a = int(round(min(dc.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW DC-Grid".format(a))
            b = int(round(min(normal.lines.s_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW AC-Grid".format(b))
        elif carrier == 'gas pipeline':
            c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW Pipeline".format(c))
        else:
            c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW Pipeline".format(c))
            d = int(round(min(retro.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW retrofitted Pipeline".format(d))

    #create empty set
    labels = set()

    #check if trace name is not in set and in list, if true enable showlegend and store in set
    for trace in fig['data']: 
        if ( not trace['name'] in labels) and (trace['name'] in labels_to_show_in_legend):
            labels.add(trace['name'])
            trace['showlegend'] = True
            #print(trace['name'])   #enable print for troubleshooting
        else:
            trace['showlegend'] = False


  

            
    return(fig)

In [ ]:
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px



# Build Dash components
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
mytitle = dcc.Markdown(children='# Kartendarstellung Netzkapazitäten')
mygraph = dcc.Graph(figure={})

#Drop Down Menu for Gid type
dropdown = dcc.Dropdown(options=['Hydrogen grid', 'Methane Grid', 'Electricity Grid'],
                        value='Hydrogen grid',  # initial value displayed when page first loads
                        clearable=False)
# Dropdown Menu for Years 
dropdown_year = dcc.Dropdown(options=planning_horizon,
                        value=planning_horizon[0],  # initial value displayed when page first loads
                        clearable=False)
# set a radio button
radio = dcc.RadioItems(
    options = [{'label': 'Europa', 'value': 'europe'},
              {'label': 'Österreich', 'value': 'AT'}], value =  'europe', labelStyle={'display': 'block'})

# Customize Layout
app.layout = dbc.Container([mytitle, dropdown, dropdown_year,radio, mygraph])

# Callback allows components to interact
@app.callback(
    Output(mygraph, component_property='figure'),
    Input(dropdown, component_property='value'),
    Input(dropdown_year, component_property='value'),
    Input(radio, component_property='value')
)
def update_graph(user_input, dropdown_year, radio):  # function arguments come from the component property of the Input
    mapping = {'Methane Grid' : 'gas pipeline',
     'Electricity Grid' : "AC",
     'Hydrogen grid' : 'H2 pipeline'}
    carrier = mapping[user_input]
    n = networks[dropdown_year]
    normal,retro,dc = get_grid_df(n,carrier)
    fig = plot_grid(normal, retro, dc, carrier, dropdown_year, radio)
    return fig  # returned objects are assigned to the component property of the Output




In [ ]:
# Run app
if __name__=='__main__':
    app.run_server( port=9969)

## Test Zeug


In [ ]:
carrier = 'gas pipeline'
year = '2030'
button = 'AT'
normal, retro, dc = get_grid_df(n = networks[year], carrier = carrier)
divisor = 1e3

#plot a choropleth map
fig = px.choropleth(map_df, geojson=map_df.geometry, 
                locations=map_df.index, color =map_df.index ,  
                height=500, color_continuous_scale=[[0, 'rgb(229,236,246)'],
                  [0.05, 'rgb(229,236,246)'],
                  [0.1, 'rgb(229,236,246)'],
                  [0.20, 'rgb(229,236,246)'],
                  [1, 'rgb(229,236,246)']],
                hover_name =  list(map_df.COUNTRY_ID) )

# This is to delete the indexes in the hover_name
fig.update_traces(customdata= np.stack(map_df.COUNTRY_ID, axis=0),
    hovertemplate = '<b>%{customdata}</b>' )

#Plot grid lines and set width, AC has to be treated specially 
if carrier == 'AC':

    for i in range(len(normal.lines.s_nom_opt)):
        fig.add_trace(
            go.Scattergeo(
                lon = [normal.buses.loc[normal.lines.bus0[i]]['x'].round(3), normal.buses.loc[normal.lines.bus1[i]]['x'].round(3)],
                lat = [normal.buses.loc[normal.lines.bus0[i]]['y'].round(3), normal.buses.loc[normal.lines.bus1[i]]['y'].round(3)],
                mode = 'lines',
                line = dict(width = (normal.lines.s_nom_opt[i] * normal.lines.s_max_pu[i])/divisor,color = 'green'), #s_nom_opt nur Termische Kap = s_nom_opt * s_max_pu für net
                hoverinfo = 'none',
                name="{} GW AC-Grid".format(int(round(normal.lines.s_nom_opt[i]/divisor)))
            )
        )
    for i in range(len(dc.links.p_nom_opt)):
        fig.add_trace(
            go.Scattergeo(
                lon = [normal.buses.loc[normal.links.bus0[i]]['x'].round(3), normal.buses.loc[normal.links.bus1[i]]['x'].round(3)],
                lat = [normal.buses.loc[normal.links.bus0[i]]['y'].round(3), normal.buses.loc[normal.links.bus1[i]]['y'].round(3)],
                mode = 'lines',
                line = dict(width = (dc.links.p_nom_opt[i])/divisor,color = 'red'),
                hoverinfo = 'none',
                opacity = 0.5,
                name="{} GW DC-Grid".format(int(round(dc.links.p_nom_opt[i]/divisor)))
            )
        )

else:

    for i in range(len(normal.links.p_nom_opt)):
        fig.add_trace(
            go.Scattergeo(
                lon = [normal.buses.loc[normal.links.bus0[i]]['x'].round(3), normal.buses.loc[normal.links.bus1[i]]['x'].round(3)],
                lat = [normal.buses.loc[normal.links.bus0[i]]['y'].round(3), normal.buses.loc[normal.links.bus1[i]]['y'].round(3)],
                mode = 'lines',
                line = dict(width = (normal.links.p_nom_opt[i])/divisor,color = 'red'),
                hoverinfo = 'none',
                name="{} GW Pipeline".format(int(round(normal.links.p_nom_opt[i]/divisor)))

            )
        )
    for i in range(len(retro.links.p_nom_opt)):
        fig.add_trace(
            go.Scattergeo(
                lon = [normal.buses.loc[retro.links.bus0[i]]['x'].round(3), normal.buses.loc[retro.links.bus1[i]]['x'].round(3)],
                lat = [normal.buses.loc[retro.links.bus0[i]]['y'].round(3), normal.buses.loc[retro.links.bus1[i]]['y'].round(3)],
                mode = 'lines',
                line = dict(width = (retro.links.p_nom_opt[i])/divisor,color = 'blue'),
                hoverinfo = 'none',
                name="{} GW retrofitted Pipeline".format(int(round(retro.links.p_nom_opt[i]/divisor)))
            )
        )

#centralize on europe

if button == 'europe':
    fig.update_geos(
        visible=False, resolution=50, 
        scope="europe",
        showcountries=True, countrycolor="Black",
        #showsubunits=True, subunitcolor="Blue"
    )
else:
    fig.update_geos(
        visible=False, resolution=50, 
        scope="europe",
        #this part is to zoom in on AT
        center=dict(lon=11, lat=40), # this part is to zone to AT
        projection_rotation=dict(lon=31, lat=31, roll=31),
        lataxis_range=[10,11], lonaxis_range=[33,41],
        showcountries=True, countrycolor="Black"
    )

#Scale Figure 
fig.update_layout(
    title=f" Grid Capacity in {year}",
    autosize=False,
    width=1000,
    height=1000,
    coloraxis_showscale=False,
    
)

#the following part is only for legend edititng

#create empty list to store legend labels
labels_to_show_in_legend = []

#as far as i understand plotly needs a geoobject to exist in the plot to be filtered in legend 
#therefor find the closest existing Capacity to 3 and 10 GW and store them in List
for s in (10,3):
    if carrier == 'AC':
        a = int(round(min(dc.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
        labels_to_show_in_legend.append("{} GW DC-Grid".format(a))
        b = int(round(min(normal.lines.s_nom_opt/divisor, key=lambda x:abs(x-s))))
        labels_to_show_in_legend.append("{} GW AC-Grid".format(b))
    elif carrier == 'gas pipeline':
        c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
        labels_to_show_in_legend.append("{} GW Pipeline".format(c))
    else:
        c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
        labels_to_show_in_legend.append("{} GW Pipeline".format(c))
        d = int(round(min(retro.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
        labels_to_show_in_legend.append("{} GW retrofitted Pipeline".format(d))

#create empty set
labels = set()

#check if trace name is not in set and in list, if true enable showlegend and store in set
for trace in fig['data']: 
    if ( not trace['name'] in labels) and (trace['name'] in labels_to_show_in_legend):
        labels.add(trace['name'])
        trace['showlegend'] = True
        #print(trace['name'])   #enable print for troubleshooting
    else:
        trace['showlegend'] = False


fig.show()


In [ ]:
for i in range(len(normal.links[normal.links.carrier == 'gas pipeline'])):
    print(normal.links[normal.links.carrier == 'gas pipeline'][i][['bus0','bus1','p_nom_opt']])

In [ ]:
carrier = 'gas pipeline'
year = '2030'
button = 'AT'
n = networks[year]
normal, retro, dc = get_grid_df(n = networks[year], carrier = carrier)

normal.links[normal.links.carrier == 'gas pipeline'].index,n.links[n.links.carrier == 'gas pipeline'].index


